In [1]:
# import os

# from dotenv import load_dotenv
# from langchain import hub
# from langchain.agents import AgentExecutor, create_react_agent
# from langchain.chains import create_history_aware_retriever, create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain_community.vectorstores import Chroma
# from langchain_core.messages import AIMessage, HumanMessage
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.tools import Tool
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# # Load environment variables from .env file
# load_dotenv()

# # Load the existing Chroma vector store
# current_dir = os.path.dirname(os.path.abspath(__file__))
# db_dir = os.path.join(current_dir, "..", "..", "4_rag", "db")
# persistent_directory = os.path.join(db_dir, "faiss_db_with_metadata")

# # Check if the Chroma vector store already exists
# if os.path.exists(persistent_directory):
#     print("Loading existing vector store...")
#     db = Chroma(persist_directory=persistent_directory,
#                 embedding_function=None)
# else:
#     raise FileNotFoundError(
#         f"The directory {persistent_directory} does not exist. Please check the path."
#     )

# # Define the embedding model
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# # Load the existing vector store with the embedding function
# db = Chroma(persist_directory=persistent_directory,
#             embedding_function=embeddings)

# # Create a retriever for querying the vector store
# # `search_type` specifies the type of search (e.g., similarity)
# # `search_kwargs` contains additional arguments for the search (e.g., number of results to return)
# retriever = db.as_retriever(
#     search_type="similarity",
#     search_kwargs={"k": 3},
# )

# # Create a ChatOpenAI model
# llm = ChatOpenAI(model="gpt-4o")

# # Contextualize question prompt
# # This system prompt helps the AI understand that it should reformulate the question
# # based on the chat history to make it a standalone question
# contextualize_q_system_prompt = (
#     "Given a chat history and the latest user question "
#     "which might reference context in the chat history, "
#     "formulate a standalone question which can be understood "
#     "without the chat history. Do NOT answer the question, just "
#     "reformulate it if needed and otherwise return it as is."
# )

# # Create a prompt template for contextualizing questions
# contextualize_q_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", contextualize_q_system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )

# # Create a history-aware retriever
# # This uses the LLM to help reformulate the question based on chat history
# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )

# # Answer question prompt
# # This system prompt helps the AI understand that it should provide concise answers
# # based on the retrieved context and indicates what to do if the answer is unknown
# qa_system_prompt = (
#     "You are an assistant for question-answering tasks. Use "
#     "the following pieces of retrieved context to answer the "
#     "question. If you don't know the answer, just say that you "
#     "don't know. Use three sentences maximum and keep the answer "
#     "concise."
#     "\n\n"
#     "{context}"
# )

# # Create a prompt template for answering questions
# qa_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", qa_system_prompt),
#         MessagesPlaceholder("chat_history"),
#         ("human", "{input}"),
#     ]
# )

# # Create a chain to combine documents for question answering
# # `create_stuff_documents_chain` feeds all retrieved context into the LLM
# question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# # Create a retrieval chain that combines the history-aware retriever and the question answering chain
# rag_chain = create_retrieval_chain(
#     history_aware_retriever, question_answer_chain)


# # Set Up ReAct Agent with Document Store Retriever
# # Load the ReAct Docstore Prompt
# react_docstore_prompt = hub.pull("hwchase17/react")

# tools = [
#     Tool(
#         name="Answer Question",
#         func=lambda input, **kwargs: rag_chain.invoke(
#             {"input": input, "chat_history": kwargs.get("chat_history", [])}
#         ),
#         description="useful for when you need to answer questions about the context",
#     )
# ]

# # Create the ReAct Agent with document store retriever
# agent = create_react_agent(
#     llm=llm,
#     tools=tools,
#     prompt=react_docstore_prompt,
# )

# agent_executor = AgentExecutor.from_agent_and_tools(
#     agent=agent, tools=tools, handle_parsing_errors=True, verbose=True,
# )

# chat_history = []
# while True:
#     query = input("You: ")
#     if query.lower() == "exit":
#         break
#     response = agent_executor.invoke(
#         {"input": query, "chat_history": chat_history})
#     print(f"AI: {response['output']}")

#     # Update history
#     chat_history.append(HumanMessage(content=query))
#     chat_history.append(AIMessage(content=response["output"]))


## React Agent with Faiss DataBase

In [2]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import Tool
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import Document
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import RetrievalQA
from langchain import hub
from pprint import pprint
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel , Field
from langchain_core.prompts import PromptTemplate , ChatPromptTemplate , SystemMessagePromptTemplate , HumanMessagePromptTemplate 
from langchain.output_parsers import PydanticOutputParser 
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Type


In [3]:
import os 
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
os.getenv("GROQ_API_KEY")
os.getenv("TAVILY_API_KEY")
os.getenv("ANTHROPIC_API_KEY")
llm=ChatGroq(model="Mixtral-8x7b-32768")

## Docstore Loading And Vector Store Integration

In [4]:

# Load environment variables from .env file
directory_path = "./documents"
text_files = [f for f in os.listdir(directory_path) if f.endswith('.txt')]

if text_files:
    all_chunks = []
    for text_file in text_files:
        file_path = os.path.join(directory_path, text_file)
        loader = TextLoader(file_path)
        documents = loader.load()
        
        # Extract text from the loaded documents
        for doc in documents:
            all_doc_texts=[]
            text = doc.page_content
            all_doc_texts.append(text)
            splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=200)
            chunks = splitter.split_text(text)
            
            # Create Document objects for each chunk
            for chunk in chunks:
                chunk_doc = Document(page_content=chunk)
                all_chunks.append(chunk_doc)

    # Extract text content from Document objects
    chunk_texts = [doc.page_content for doc in all_chunks]

    model_name = "BAAI/bge-small-en"
    model_kwargs = {"device": "cpu"}
    encode_kwargs = {"normalize_embeddings": True}
    hf = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
    
    # Create FAISS vector store
    vectordb = FAISS.from_texts(chunk_texts, embedding=hf)
    vectordb.save_local("./db_index")
    load_vectordb = vectordb.load_local("./db_index", embeddings=hf, allow_dangerous_deserialization=True)
    retriever = load_vectordb.as_retriever(search_kwargs={"k": 1})


c:\Users\abdul\OneDrive\Desktop\Langchain learning\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\abdul\OneDrive\Desktop\Langchain learning\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
Agent_Prompt=""""Answer the following questions as best you can. You have access to the following tools:

{tools}
One Important Note!!! : After using the 'Answer Question' tool, always use the 'Grade and Improve Retrieval' tool to ensure the information is relevant. If the 'Grade and Improve Retrieval' tool returns new information, incorporate it into your answer.
Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}"""

### Doc_store_retriever

In [6]:


# Contextualize question prompt
# This system prompt helps the AI understand that it should reformulate the question
# based on the chat history to make it a standalone question
contextualize_q_system_prompt = (
    """You are zilbils customer sevice chatbot which is a clothing brand Your task is that Given a chat history and the latest user question 
    which might reference context in the chat history, 
    formulate a standalone question which can be understood 
    without the chat history. And then from the documents provided to you give the most suitable answer to that question 
    """
)

# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question prompt
# This system prompt helps the AI understand that it should provide concise answers
# based on the retrieved context and indicates what to do if the answer is unknown
qa_system_prompt = (
    "You are zilbils customer sevice chatbot which is a clothing brand . Use "
    "the following pieces of retrieved context to answer the "
    "question.If you know the generate answer that politely clear's the doubts and their question .If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)

# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# `create_stuff_documents_chain` feeds all retrieved context into the LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create a retrieval chain that combines the history-aware retriever and the question answering chain
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain)


In [7]:

class Grade_Improve(BaseModel):
    """It is used to grade the information retrived from docstore against the user_query and search for more relevant info if needed."""
    query: str = Field(description="The user query")
    retrived_info: str = Field(description="Information Retrived from DocStore")

class grade_and_improve_retrieval(BaseTool):
    name = "Multiply numbers"
    description = "Useful for multiplying two numbers"
    args_schema: Type[BaseModel] = Grade_Improve

    def _run(self, query: str, retrieved_info: str) -> str:
        """
        Grade the retrieved information and search for more relevant info if needed.

        Args:
            query: str = The user query
            retrieved_info: str = The information retrieved from the docstore

        Returns:
            str = The graded and improved retrieved information
        """

        class GradeHallucination(BaseModel):
            hallucinate_score: str = Field(description="Is the retrieved_information hallucinated, Answer with 'Yes' OR 'No'")

        hallucinator_parser = PydanticOutputParser(pydantic_object=GradeHallucination)

        hallucinator_system_prompt = """You are a grader Accessor whether an LLM generation is supported by a set of retrieved facts.
        \n Restrict yourself to give a binary score of either a 'Yes' or 'No'. If the answer is supported partially by the set of facts, consider it a yes.
        \n Don't consider calling external APIs for additional Information as consistent with facts"""

        hallucinator_human_template = "{query} \n {retrieved_info} \n{formatted_output}"

        chat_prompt_template = ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template(hallucinator_system_prompt), HumanMessagePromptTemplate.from_template(hallucinator_human_template)])
        formatting_template = chat_prompt_template.format_messages(input=query, retrieved_info=retrieved_info, formatted_output=hallucinator_parser.get_format_instructions())

        grade_response = llm.invoke(formatting_template)

        if grade_response.content == 'Yes':
            return retrieved_info
        elif grade_response.content == 'No':
            # Search for more relevant information
            search_prompt = f"Find relevant information for the query: " + query

            zilbils_retrieval_prompt = (
                """You are zilbils customer service chatbot which is a clothing brand. Your task is that Given a chat history and the latest user question
                which might reference context in the chat history,
                formulate a standalone question which can be understood
                without the chat history. And then from the documents provided to you give the most suitable answer to that question"""
            )

            def format_docs(documents):
                return "\n\n".join(doc.page_content for doc in documents)

            hallucinate_rag_chain = (
                {"context": retriever | format_docs, "question": RunnablePassthrough()}
                | zilbils_retrieval_prompt
                | llm
                | StrOutputParser()
            )

            hallucinate_rag_chain.invoke(search_prompt)

        return f"The new retrieved Information is {hallucinate_rag_chain}"

hallucinate_tool=grade_and_improve_retrieval()
#hallucinator removal checker

# def grade_and_improve_retrieval(query: str, retrieved_info: str) -> str:
#     """
#     Grade the retrieved information and search for more relevant info if needed.
#     Args : 
#         query : str = The user query
#         retrieved_info : str = The information retrived from the docstore
#     Returns :
#         str = The graded and improved retrieved information
#     """



    
#         return f"Original retrieval was not sufficiently relevant. Here's new information: {hallucinate_rag_chain}"


# grade_improve_tool = StructuredTool.from_function(
#     func=grade_and_improve_retrieval,
#     name="DocStore Retrived Information Grader",
#     description="Must Use Tool After Question Answer Tool , It is used to grade the information retrived from docstore against the user_query and retrieve for more relevant info if needed.",
#     args_schema=Grade_Improve,
#     return_direct=False,
#     # handle_tool_error=_handle_error

#     # coroutine= ... <- you can specify an async method if desired as well
# )



In [8]:
react_prompt = """Answer the following questions as best you can , 1st Important Note!!! : After using the 'Answer Question' tool, always use the 'Grade and Improve Retrieval' tool to ensure the information is relevant. If the 'Grade and Improve Retrieval' tool returns new information, incorporate it into your answer . 
2nd Important Note!!!: Your Final Answer should never say that 'you must contact the customer support for further clarification' , yu have to clarify the user_input by giving them the relevant answer and dont give a final answer that is saying contact this or that because you are the representative of a company \n . 
You have access to the following tools:
{tools}

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}"""



In [9]:
from langchain.prompts import PromptTemplate
react_docstore_prompt=PromptTemplate.from_template(react_prompt)

In [10]:
# Set Up ReAct Agent with Document Store Retriever
# Load the ReAct Docstore Prompt
# react_docstore_prompt = hub.pull("hwchase17/react")


search = TavilySearchResults()
def websearch_tool(web_input:str)->str:
    """This will search the user query on the internet if it is not related to zilbil and its policies"""
    custom_content=[]
    web_response=search.run(web_input)
    for result in web_response:
        web_content=result.get('content' , '')
        if web_content:
            custom_content.append(web_content)
    return custom_content


from datetime import datetime


date_tool = Tool(
    name="Date",
    func=lambda x: datetime.now().strftime("%d - %m - %Y"),
    description="Returns the current date in day - month - year format"
)



time_tool = Tool(
    name="Time",
    func=lambda x: datetime.now().strftime("%H:%M"),
    description="Returns the current time in hour:minute format"
)



tools = [
    # hallucinate_tool,
    search,
    date_tool,
    time_tool,
    # Tool(name="Web Search Engine" , func=websearch_tool , description="Use when the question is not about zilbil our clothing brand, use it to search the internet") , 
    Tool(
        name="Answer Question",
        func=lambda input, **kwargs: rag_chain.invoke(
            {"input": input, "chat_history": kwargs.get("chat_history", [])}
        ),
        description="Useful if you need to answer questions related to zilbil and its products",
    ),
    # Tool(
    #     name="Grade and Improve Retrieval",
    #     func=grade_and_improve_retrieval,
    #     description="Must Use this to grade the relevance of retrieved information after retriving information from vectorstore",
    # )
]

In [12]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_docstore_prompt,
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True,
)

chat_history = []
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    response = agent_executor.invoke(
        {"input": query, "chat_history": chat_history})
    print(f"AI: {response['output']}")

    # Update history
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=response["output"]))




> Entering new AgentExecutor chain...
The user is asking for the current date. I can use the 'Date' tool to get this information.

Action: Date
Invalid Format: Missing 'Action Input:' after 'Action:'It seems that the 'Date' tool requires an input, even though it's not necessary in this case. I'll just provide a random string as input so that the tool works.

Action: Date('dummy input')
Invalid Format: Missing 'Action Input:' after 'Action:'It appears that the 'Date' tool is not functioning as expected. I will try using the 'Time' tool instead, as it might provide the current date as well.

Action: Time
Invalid Format: Missing 'Action Input:' after 'Action:'The 'Time' tool also requires an input, and it doesn't seem to provide the current date. I will try using the tavily\_search\_results\_json tool to search for the current date.

Action: tavily\_search\_results\_json(current date)
Invalid Format: Missing 'Action Input:' after 'Action:'It seems that the format provided for using the 

KeyboardInterrupt: 